## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,East London,ZA,2022-07-23 21:28:47,broken clouds,-33.0153,27.9116,57.63,71,72,18.19
1,1,Asau,RO,2022-07-23 21:32:48,broken clouds,46.4333,26.4000,75.96,73,52,2.89
2,2,Talnakh,RU,2022-07-23 21:32:48,broken clouds,69.4865,88.3972,63.21,60,81,4.70
3,3,Ketchikan,US,2022-07-23 21:29:05,broken clouds,55.3422,-131.6461,59.47,77,75,16.11
4,4,Namtsy,RU,2022-07-23 21:32:49,clear sky,62.7161,129.6658,68.18,67,5,7.78


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Asau,RO,2022-07-23 21:32:48,broken clouds,46.4333,26.4000,75.96,73,52,2.89
9,9,Puerto Ayora,EC,2022-07-23 21:32:50,broken clouds,-0.7393,-90.3518,78.76,89,81,5.01
11,11,Ribeira Grande,PT,2022-07-23 21:32:51,scattered clouds,38.5167,-28.7000,75.60,88,40,10.36
15,15,Faanui,PF,2022-07-23 21:32:52,clear sky,-16.4833,-151.7500,79.56,73,9,12.91
17,17,Maine-Soroa,NE,2022-07-23 21:32:53,scattered clouds,13.2114,12.0241,85.35,50,49,5.97


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 1 to 17
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              5 non-null      int64  
 1   City                 5 non-null      object 
 2   Country              5 non-null      object 
 3   Date                 5 non-null      object 
 4   Current Description  5 non-null      object 
 5   Lat                  5 non-null      float64
 6   Lng                  5 non-null      float64
 7   Max Temp             5 non-null      float64
 8   Humidity             5 non-null      int64  
 9   Cloudiness           5 non-null      int64  
 10  Wind Speed           5 non-null      float64
dtypes: float64(4), int64(3), object(4)
memory usage: 480.0+ bytes


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()
clean_df.dropna(how="any",inplace=True)
clean_df

,City_ID,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Asau,RO,2022-07-23 21:32:48,broken clouds,46.4333,26.4000,75.96,73,52,2.89
9,9,Puerto Ayora,EC,2022-07-23 21:32:50,broken clouds,-0.7393,-90.3518,78.76,89,81,5.01
11,11,Ribeira Grande,PT,2022-07-23 21:32:51,scattered clouds,38.5167,-28.7000,75.60,88,40,10.36
15,15,Faanui,PF,2022-07-23 21:32:52,clear sky,-16.4833,-151.7500,79.56,73,9,12.91
17,17,Maine-Soroa,NE,2022-07-23 21:32:53,scattered clouds,13.2114,12.0241,85.35,50,49,5.97


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Asau,RO,75.96,broken clouds,46.4333,26.4000,
9,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
11,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,
15,Faanui,PF,79.56,clear sky,-16.4833,-151.7500,
17,Maine-Soroa,NE,85.35,scattered clouds,13.2114,12.0241,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
     "type": "lodging",
     "key": g_key,
    
 }
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Asau,RO,75.96,broken clouds,46.4333,26.4000,Casa Mika Ciobanus
9,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
15,Faanui,PF,79.56,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
17,Maine-Soroa,NE,85.35,scattered clouds,13.2114,12.0241,Complexe


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
# params = {
#     "radius": 5000,
#     "type": "lodging",
#     "key": g_key
# }

# # 6b. Iterate through the hotel DataFrame.
# for index, row in hotel_df.iterrows():


#     # 6c. Get latitude and longitude from DataFrame
#     lat = row["Lat"]
#     lng = row["Lng"]
#     # Add the latitude and longitude to location key for the params dictionary.
#     params["location"] = f"{lat},{lng}"

    

    
#     # 6d. Set up the base URL for the Google Directions API to get JSON data.
#     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    

#     # 6e. Make request and retrieve the JSON data from the search. 
#     hotels = requests.get(base_url, params=params).json()


    
   
    
#     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
#     #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
#     # Grab the first hotel from the results and store the name.
#     try:
#         hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
#     except (IndexError):
#         print("Hotel not found... skipping.")

    
    
    
    
        

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Asau,RO,75.96,broken clouds,46.4333,26.4000,Casa Mika Ciobanus
9,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
15,Faanui,PF,79.56,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
17,Maine-Soroa,NE,85.35,scattered clouds,13.2114,12.0241,Complexe


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "./Weather_Database/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
#check to get map working
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#11a add a marker layer for each city on the map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b display the figure
fig




Figure(layout=FigureLayout(height='420px'))